preprocess the binance data to only consist of ohlcv features and save it to a csv file

we also had to convert the timestamp as it was mixed up with ms and s

source: https://data.binance.vision/?prefix=data/spot/monthly/klines/BTCUSDT/1m/

In [2]:
import os
import pandas as pd
from datetime import datetime


In [ ]:
# Directory path
dir_path = "./trading_envs/data/binance_btc_1m_raw/"
columns = ["open time", "open", "high", "low", "close", "volume", "close time", "quote asset volume", "number of trades", "taker buy base asset volume", "taker buy quote asset volume", "ignore"]

In [4]:

# Get list of relevant CSV files
files = [f for f in os.listdir(dir_path) if f.startswith("BTCUSDT-1m-") and f.endswith(".csv")]

# Extract dates and pair with filenames
file_dates = []
for f in files:
    date_str = f.split("-1m-")[1].replace(".csv", "")
    date = datetime.strptime(date_str, "%Y-%m")
    file_dates.append((f, date))

# Sort by date in ascending order (historical order)
sorted_files = sorted(file_dates, key=lambda x: x[1])

# Load and concatenate DataFrames
dfs = []
for f, _ in sorted_files:
    df = pd.read_csv(os.path.join(dir_path, f), header=None, usecols=[0, 1, 2, 3, 4, 5]) # only open, high, low, close, volume
    dfs.append(df)

# Concatenate all DataFrames
full_df = pd.concat(dfs, ignore_index=True)
# adapt time stamp error

# Function to convert timestamps
def convert_timestamp(value):
    try:
        # Convert to float to handle numeric inputs
        value = float(value)
        # Check if timestamp is in microseconds (unusually large, e.g., > 10^12)
        if value > 1_000_000_000_000_000:  # Likely microseconds
            return pd.to_datetime(value / 1_000_000, unit='s')  # Convert µs to seconds
        # Assume milliseconds for reasonable range (2020-2030)
        elif 1_600_000_000_000 <= value <= 1_800_000_000_000:  # Approx 2020-2030 in ms
            return pd.to_datetime(value, unit='ms')
        # Try seconds if in reasonable range
        elif 1_600_000_000 <= value <= 1_800_000_000:  # Approx 2020-2030 in seconds
            return pd.to_datetime(value, unit='s')
        else:
            return pd.NaT  # Invalid timestamp
    except (ValueError, TypeError):
        return pd.NaT  # Non-numeric or invalid

full_df[0] = full_df[0].apply(convert_timestamp)


In [5]:
full_df

,0,1,2,3,4,5
0,2024-12-01 00:00:00,96407.99,96411.99,96376.00,96376.00,8.18452
1,2024-12-01 00:01:00,96376.00,96384.76,96350.13,96350.13,6.72047
2,2024-12-01 00:02:00,96350.13,96411.99,96350.13,96411.99,3.50598
3,2024-12-01 00:03:00,96411.99,96491.58,96408.01,96491.58,5.55435
4,2024-12-01 00:04:00,96491.57,96524.00,96491.57,96508.29,11.91174
...,...,...,...,...,...,...
394555,2025-08-31 23:55:00,108232.18,108275.32,108226.33,108275.32,6.72961
394556,2025-08-31 23:56:00,108275.31,108275.32,108226.02,108264.25,10.89686
394557,2025-08-31 23:57:00,108264.25,108264.25,108247.31,108247.31,3.65510
394558,2025-08-31 23:58:00,108247.32,108247.32,108214.30,108214.31,5.14475


In [7]:
full_df.to_csv("./btcusdt_spot_1m_12_2024_to_09_2025.csv", index=False)